## TOC:
* [Abstract](#abs)
* [Introduction](#intro)
* [Exact Riemann Solver](#ers)
    * [Riemann Problem](#ers-rp)
    * [Solution of Riemann Problem](#ers-so)
    * [Tests](#ers-te)
        * [Sod (shock tube) test](#ers-te-so)
        * [123 test](#ers-te-123)
        * [Left blast wave test](#ers-te-lbw)
        * [Right blast wave test](#ers-te-rbw)
        * [Two shocks test](#ers-te-tw)
    * [Data Visualisation](#ers-dv)
* [Machine Learning Powered Riemann Solver](#mlrs)
    * [Range of Interest](#mlrs-ri)
    * [Models](#mlrs-mo)
    * [Tests](#mlrs-te)
* [Conclusion](#con)
* [References](#refer)
* [Appendix I: Derivation of pressure function for shocks and rarefactions](#app-1)

In [ ]:
# global packages and variables
from ml_riemann_solver import MLRiemannSolver as MLRS

import plotly.express as px

import os
from pathlib import Path
import pickle

# Backup directory
Path(os.environ['HOME'] + '/.mlrs').mkdir(parents=True, exist_ok=True)

## Abstract <a class="anchor" id="abs"></a>

## Introduction <a class="anchor" id="intro"></a>

[Godunov 1959](#ref-godunov-1959) introduced a conservative scheme for numerically solving non-linear systems of hyperbolic conervation laws. His original 1st-order scheme three main steps:
* Data reconstruction
* Solving local Riemann problem at cell boundaries
* Evolving hydro variables

Since 1959, higher-order schemes were invented by varius people based on the Godunov scheme (references).

Briefly mention higher-order TVD schemes:
* [ ] Wighted average flux (WAF) schemes
* [ ] MUSCL-Hancock scheme

The key ingredient of all these schemes is the solution of local Riemann problem.

Briefly mention and reference different Riemann solvers here:
* [ ] HLL and HLLC solvers
* [ ] Roe solver
* [ ] Approximate-state solvers
* [ ] Exact solver (more information in a separate chapter)

In a typical hydrodynamical simulations, Riemann problem must be solved at all cell boundaries for every time step, thus any improvement in the performance of this task will lead to a significant improvement in overall performance of hydrodynamical simulations.

The purpose of this study is to introduce an accurate Machine Learning powered Riemann Solver (MLRS hearby) with O(1) complexity to be substitute by current solvers. Our method consist of a multi-layer netwrork trained on a sample of Riemann problem solutions generated by the exact Riemann solver within a wide range of hydrodynamical varibales. Since we are specifically interested in the range of variables relevant for astrophisycal simulations, we limit our scope of interest to only ideal gasses and the solution of the Riemann problem at its initial boundary.

## Exact Riemann Solver <a class="anchor" id="ers"></a>

Riemann problem does not have a closed-form solution and what we refer here as the exact Riemann solver is an iterative scheme that can be used to calculate the solution of a Riemann problem with a desired accuracy. Here, we are interested in solution of Riemann problem for ideal gasses at the initial boundary of two states (look at [Colella and Glaz 1985](#ref-colella-1985) for a discussion about general Riemann solvers).

The exact solution of the Riemann problem is an important reference solution to be used to asses the accuracy and performance of numerical simulations and methods. It is also a good example of a simple intial condition which leads to a rich non-trivial result containing the mathematical and physical essensce of the conservation laws.

The solution of the Riemann problem can be seen as the non-linear superposition of solutions of local Riemann problems [Glimm 1965](#ref-glimm-1965).

In this chapter we present a procedure to iteratively solve the Riemann problem of inviscid compressible flow.

### Riemann Problem <a class="anchor" id="ers-rp"></a>

A Riemann problem is an Initial Value Problem (IVP) composed of conservation laws together with intiial piecewise constant data having a discontinuity at the centre.

In Cartesian space, conservation laws can be described as,

$$\partial_t U + \partial_x F(U) = 0$$

where $U$ is a vector of conservative variable and $F(U)$ is their fluxes. In three-dimensional space $U$ can be written as,

$$U = \begin{bmatrix} \rho \\ \rho u \\ \rho v \\ \rho w \\ E\end{bmatrix}$$

where $u$, $v$ and $w$ are velocities in $x$, $y$ and $z$ directions and $E$ is the total energy consists of kinetic and internal energies and it equals to $\frac{1}{2} \rho \mathbf{v}^2 + e_{\text{int}}$. The corresponding flux of the flow in $x$ direction can be written as:

$$F_x(U) = \begin{bmatrix} \rho u \\ \rho u^2 + p \\ \rho u v \\ \rho u w \\ u(E + p) \end{bmatrix}$$

where $p$ is the pressure and can be calculated by the caloric ideal gas Equation of State (EoS):

$$e_{\text{int}} = \frac{p}{(\gamma - 1) \rho}$$

The initial piecewise constant condition is also written as,

$$U(x,t=0) = U^{(0)}(x) = \begin{cases}
U_L & \text{if} \quad x < 0 \\
U_R & \text{if} \quad x > 0
\end{cases}$$

Where $U_L$ and $U_R$ are vectors of conservative variables and have constant values over the domain of interest.

Here, we assume that the system is strictly hyperbolic, meaning the
eigenvalues of the Jacobian matrix $\mathbf{A} = \partial_{u_j} f_i$ are
real and distinct,

$$\lambda_1 < \lambda_2 < ... < \lambda_m$$

Each eigenvector corresponds to a wave emerging from the discontinuity and propagating with the velocity equals to its eigenvalue. Generally, the solution of the Riemann problem consists of three types of waves,

* one linearly degenerate wave (the contact discontinuity)
* (nonlinear) Shock waves
* (nonlinear) rarefaction (fan) waves

Contact discontinuities are surfaces that separate zones of different density and temperature at fixed pressure. By definition such a surface is in pressure equilibrium, and no gas flows across it. Usually, when the tangential components of velocity on one side differs considerably from that of the gas on the other side, the surface is called a slip discontinuity. The boundary of a supersonic jet and the ambient gas is an example of a slip discontinuity.

The two types of nonlinear waves (shocks and rarefactions) arise from abrupt changes in pressure. Shock fronts accompany compression of the medium and rarefactions accompany expansion of the medium.

These waves are separating four different states where the conservative vector $U$ acquires from the left to the right the following values, $U_L$, $U_{*L}$, $U_{*R}$ and $U_R$. The symbol $*$ identify points located in the star region between the nonlinear waves.

### Solution of Riemann Problem <a class="anchor" id="ers-so"></a>

The solution for the pressure at contact discontinuity of the Riemann problem for an ideal gas, $p_*$, is given by the root of the following equation, called the **pressure function**,

$$
f(p, W_L, W_R) \equiv f_L(p,W_L) + f_R(p, W_R) + \Delta u = 0,
\quad \Delta u \equiv u_R - u_L
$$

where \$f_K$ for $k \in [L, R]\$ represents relations across the left and the right nonlinear waves and is given by:

$$
f_K \left(p, W_K\right) = \begin{cases}
  (p - p_K) \sqrt{(\frac{A_K}{p + B_K})}
  & \text{if}\quad p > p_K \quad \text{shock} \\
  & \\
  \frac{2 c_{sK}}{\gamma - 1} \left[
    \left( \frac{p}{p_K} \right)^{\frac{\gamma - 1}{2 \gamma}} - 1
  \right]
  & \text{if}\quad p < p_K \quad \text{rarefaction}
\end{cases}
$$

$$
A_K = \frac{2}{(\gamma + 1) \rho_K}, \quad
B_K = \frac{\gamma - 1}{\gamma + 1} p_K
$$

Following the calculation of $p_*$, the velocity at contact discontinuity, $u_*$, which is a function of the left and right velocities and $p_*$, can be calculated as,

$$u_* = \frac 1 2 (u_L + u_R) + \frac 1 2 \left[ f_R(p_*) - f_L(p_*) \right]$$

The remaining unknowns can be found by using standard gas dynamics relations.

In order to investigate the behaviour of the pressure function at a given data ($u_L = (\rho_L, u_L, v_L, w_L, p_L)^T$ and $u_R = (\rho_R, u_R, v_R, w_R, p_R)^T$), we need to calculate the first and the second derivative of the function w.r.t $p$. The first derivative of the pressure function can be written as,

$$
f^{\prime}_K =
\begin{cases}
  (\frac{A_K}{p + B_K})^{1/2}
  \left(
    1 - \frac{p - p_K}{2 (B_k + p)}
  \right) & \text{if}\quad p > p_K \quad \text{shock} \\
  & \\
  \frac{1}{\rho_K c_s^K}
  \left(
    \frac{p}{p_K}
  \right)^{\frac{- (\gamma + 1)}{2 \gamma}}
  & \text{if}\quad p \lt p_K \quad \text{rarefaction}
\end{cases}
$$

as \$f^\prime = f_L^\prime + f_R^\prime$ and both $f_{L,R}^\prime > 0$, we can see that the pressure function is monotonic function.

Considering the second derivative of the pressure function

$$
f^{\prime\prime}_K =
\begin{cases}
\frac 1 4 \left( \frac{ A_K }{ p + B_K } \right)^{1/2}
\left[
  \frac{ 4B_K + 3p + p_K }{ ( B_K + p )^2 }
\right]
& \text{if}\quad p > p_K \quad \text{shock} \\
& \\
- \frac{ (\gamma - 1) c_{sK} }{ 2 \gamma^2 p_K^2 }
\left(
  \frac{ p }{ p_K }
\right)^{ - ( 3\gamma + 1 ) / 2\gamma }
& \text{if}\quad p \leq p_K \quad \text{rarefaction}
\end{cases}
$$

since $f^{\prime\prime} = f_l^{\prime\prime} + f_r^{\prime\prime}$ and both $f_{L,R}{^{\prime\prime} < 0$, the pressure function is downward concave.

The following figure shows the behaviour of the pressure function in the solution of the Riemann problem,

![pressure-function](assets/pressure-function.png)

In the above figure, we define,

$$
\begin{aligned}
  & p_{\text{min}} = \text{min}(p_L, p_R) \\
  & p_{\text{max}} = \text{max}(p_L, p_R) \\
  & f_{\text{min}} = f(p_{\text{min}}) \\
  & f_{\text{max}} = f(p_{\text{max}})
\end{aligned}
$$

For a given \$p_L$ and $p_R$, it is the velocity differece of two sides, $\Delta u$, that determines the value of $p_*$.

Three distinct intervals, $I_1$, $I_2$ and $I_3$, can be identified in the domain of the pressure function:

$$
\begin{aligned}
  & p_*\quad\text{lies in}\quad I_1 = (0, p_{\text{min}})\quad
  &\text{if}\quad f_{\text{min}} > 0\quad\text{and}\quad f_{\text{max}} > 0\\
  & p_*\quad\text{lies in}\quad I_2 = [p_{\text{min}}, p_{\text{max}}]\quad
  &\text{if}\quad f_{\text{min}} \leq 0\quad\text{and}\quad f_{\text{max}} \geq 0\\
  & p_*\quad\text{lies in}\quad I_3 = (p_{\text{max}}, \infty)\quad
  &\text{if}\quad f_{\text{min}} < 0\quad\text{and}\quad f_{\text{max}} < 0
\end{aligned}
$$

For suficiently large \$\Delta u$ as in $(\Delta u)_1$, the value of $p_*$ lies in the $I_1$ region and since the root of the pressure function is less than $p_L$ and $p_R$, we end up having two rarefactions. We can follow the same logic to explain $(\Delta u)_2$ and $(\Delta u)_3$.

Note for a positive solution of star region pressure $p_*$ we require $f(0) < 0$. This leads to the pressure positivity condition,

$$
(\Delta u)_{\text{crit}}
  \equiv \frac{2 c_{sL}}{\gamma -1} + \frac{2 c_{sR}}{\gamma -1}
  > u_R - u_L
$$

Note that vacuum is created by the nonlinear waves if this condition is violated.

Add a new subsection for the solution of Riemann problem in the presense of vacuum:
* at the right side
* at the left side
* at both sides

### Tests <a class="anchor" id="ers-te"></a>

#### Sod (shock tube) test <a class="anchor" id="ers-te-so"></a>

#### 123 test <a class="anchor" id="ers-te-123"></a>

#### Left blast wave test <a class="anchor" id="ers-te-lbw"></a>

#### Rigth blast wave test <a class="anchor" id="ers-te-rbw"></a>

#### Two shocks test <a class="anchor" id="ers-te-tw"></a>

### Data Visualisation <a class="anchor" id="ers-dv"></a>

## Machine Learning Powered Riemann Solver <a class="anchor" id="mlrs"></a>

### Range of Interest <a class="anchor" id="mlrs-ri"></a>

### Models <a class="anchor" id="mlrs-mo"></a>

### Tests <a class="anchor" id="mlrs-te"></a>

## Conclusion <a class="anchor" id="con"></a>

## References <a class="anchor" id="refer"></a>
* [A Finite Difference Method for the Computation of Discontinuous Solutions of the Equations of Fluid Dynamics](http://www.mathnet.ru/links/ece056bd10695ed71d90b68f97f15e20/sm4873.pdf) by S. K. Godunov [1959] <a class="anchor" id="ref-godunov-1959"></a>
* [Solution in the Large for Nonlinear Hyperbolic Systems of Equations](https://onlinelibrary.wiley.com/doi/epdf/10.1002/cpa.3160180408) by J. Glimm [1965] <a class="anchor" id="ref-glimm-1965"></a>
* [Efficient Solution Algorithms for the Riemann Problem for Real Gases](https://www.sciencedirect.com/science/article/pii/0021999185901469) by P. Colella and H. H. Glaz [1985] <a class="anchor" id="ref-colella-1985"></a>

## Appendix I: Derivation of pressure function for shocks and rarefactions <a class="anchor" id="app-1"></a>

In order to derive the pressure function for shock waves we move to a frame of reference moving at the shock velocity and we introduce the relative velocities as,

$$\hat{u}_K = u_K - S_K, \quad \hat{u}_* = u_* - S_K$$

By substituting Rankine-Hugoniot conditions in to $f_K$ equation we can derive a relation for shock waves as,

$$
\begin{aligned}
  \rho_K \hat{u}_K &= \rho_* \hat{u}_* \\
  \rho_K \hat{u}_K^2 + p_K &= \rho_* \hat{u}_*^2 + p_* \\
  \hat{u}_K (\hat{E}_K + p_K) &= \hat{u}_* (\hat{E}_* + p_*)
\end{aligned}
$$

* [ ] Substitution?

In the case of a rarefaction wave, $f_K$ can be derived by assuming the isentropic relation

$$p = C \rho^{\gamma}, \quad C = p_K / \rho_K^{\gamma}$$

from which we write,

$$\rho_{*K} = \rho_K \left( \frac{p_*}{p_K} \right)^{\frac 1 \gamma}$$

and Generalised Riemann Invariant

$$I_K(u, c_s) = u_K + \frac{2 c_{sK}}{\gamma - 1}$$

which is constant across a rarefaction wave, so we can write,

$$
\begin{aligned}
  & u_L + \frac{2 c_{sL}}{\gamma - 1} = u_* + \frac{2 c_{s*L}}{\gamma - 1} \\
  & u_* - \frac{2 c_{s*R}}{\gamma - 1} = u_R - \frac{2 c_{sR}}{\gamma - 1}
\end{aligned}
$$

where,

$$
c_{s*K} = c_{sK} \left(
  \frac{p_*}{p_K}
\right)^{\frac{\gamma - 1}{2 \gamma}}
$$